In [5]:
import serial
import time
import mysql.connector
import pandas as pd

In [2]:
arduino = serial.Serial(
    port='COM6',
    baudrate=115200,
    )

SerialException: could not open port 'COM6': FileNotFoundError(2, 'O sistema não pode encontrar o arquivo especificado.', None, 2)

In [7]:
mysql = mysql.connector.connect(
    host="localhost",
    database = "exercicio_php",
    port=3306,
    user="root",
    password="1234")

In [29]:
def readSerial():
    message = ''
    while (arduino.inWaiting() > 0):
        message += arduino.readline(1).decode()
    
    if ("nodeNumber" in message):
        print(message)
        
        nodeNumber = getNodeNumber(message)
        nodeStatus = getNodeStatus(message)
        
        dados = pd.read_sql(
        sql = "SELECT * FROM mesh_network WHERE node = " + str(nodeNumber) + " ",
        con = mysql
        )

        if dados.empty:
            sql = "insert into mesh_network VALUES( " + str(nodeNumber) + ", " + str(nodeStatus) + " )"
            print("insert")
        else:
            sql = "update mesh_network SET state = " + str(nodeStatus) + " WHERE node = " + str(nodeNumber) +" "
            print("update")
            
        mycursor = mysql.cursor()
        mycursor.execute(sql)
        mysql.commit()

In [5]:
def getNodeNumber(message):
    node_index = message.find("nodeNumber")
    starting_index = node_index + 12
    final_index = starting_index + 1

    nodeNumber = message[starting_index:final_index]

    return nodeNumber

In [6]:
def getNodeStatus(message):
    node_index = message.find("state")
    starting_index = node_index + 7
    final_index = starting_index + 1

    state = message[starting_index:final_index]

    if (state == 't'):
        state = 1
    else:
        state = 0

    return state

In [ ]:
while (True):
    readSerial()
    time.sleep(0.01)


setLogLevel: ERROR | STARTUP |
STARTUP: init(): 0
STARTUP: AP tcp server established on port 5555
{"nodeNumber":1,"state":true}{"nodeNumber":1,"state":true}
{"nodeNumber":1,"state":true}
{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"st
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1,"state":true}
{"nodeNumber":3,"state":true}

{"nodeNumber":2,"state":true}

{"nodeNumber":1
{"nodeNum

In [30]:
nodeNumber = 4
nodeStatus = 1

dados = pd.read_sql(
    sql = "SELECT * FROM mesh_network WHERE node = " + str(nodeNumber) + " ",
    con = mysql
)


if dados.empty:
    sql = "insert into mesh_network VALUES( " + str(nodeNumber) + ", " + str(nodeStatus) + " )"
    print("insert")
else:
    sql = "update mesh_network SET state = " + str(nodeStatus) + " WHERE node = " + str(nodeNumber) +" "
    print("update")
mycursor = mysql.cursor()
mycursor.execute(sql)
mysql.commit()

insert
